# Create Tables

This notebook provides an example for creating databases, schemas, tables, and columns using the Secoda API. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

## Configure `requests` with your API key

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [14]:
import requests

API_KEY = ""
SECODA_API_URL = "https://api.secoda.co"

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))

def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

## Create a cluster

Secoda organizes in a Cluster -> Database -> Schema -> Table -> (Column, Query, Test) hierarchy. The first step is to create a cluster.

In [8]:
# first make sure no clusters with the same name exist to avoid inserting duplicate entries
my_cluster_name = "bigquery"
clusters = session.get(
    build_url(f"/table/clusters?title={my_cluster_name}")
).json().get("results", [])

# create a cluster if non exist with the name
if len(clusters) == 0:
    cluster = session.post(build_url("/table/clusters/"), json=dict(
        title=my_cluster_name,
    )).json()
else:
    cluster = clusters[0]

## Create a database

Once your cluster is created, the next step is to create a database in your cluster. For this one we need to make sure a cluster is already created and use its `id` to create a relationship between the two.

In [9]:
databases = session.get(
    build_url(f"/table/databases?title=my-database&parent_id={cluster['id']}")
).json().get("results", [])

if len(databases) == 0:
    database = session.post(build_url("/table/databases/"), json=dict(
        title="my-database",
        parent=cluster["id"]
    )).json()
else:
    database = databases[0]

## Create a schema

In this step we will create a schema and assign it to the database from previous step.

In [10]:
schemas = session.get(
    build_url(f"/table/schemas?title=public&parent_id={database['id']}")
).json().get("results", [])

if len(schemas) == 0:
    schema = session.post(build_url("/table/schemas/"), json=dict(
        title="public",
        parent=database["id"]
    )).json()
else:
    schema = schemas[0]

## Create a table

Now finally the time to create the tables on Secoda in the schema we created in the previous step

In [11]:
tables = session.get(
    build_url(f"/table/tables?title=customers&parent_id={schema['id']}")
).json().get("results", [])

if len(tables) == 0:
    table = session.post(build_url("/table/tables/"), json=dict(
        title="customers",
        parent=schema["id"],
        schema=schema["title"],
        database=database["title"],
        cluster=cluster["title"],
        description="customers table generated using the secoda api",
        definition="# markdown documentation for table\n this shows up in the documentation tab"
    )).json()
else:
    table = tables[0]

## Create a column

We are almost done. Just have to create columns on the table we have created

In [12]:
columns = session.get(
    build_url(f"/table/columns?title=customer_id&parent_id={table['id']}")
).json().get("results", [])

if len(columns) == 0:
    column = session.post(build_url("/table/columns/"), json=dict(
        title="customer_id",
        parent=table["id"],
        description="a unique identifier",
        type="int64",
        is_pk=True,
        sort_order=0
    )).json()
else:
    column = columns[0]

## Wrapping Up

Finally, open the table you just created in Secoda to test it!

In [13]:
f"https://app.secoda.co/table/{table['id']}"

'https://app.secoda.co/table/dd975ccd-313c-448a-8528-5e7e82843343'